In [1]:
import json
import os
import re
from functools import cache
from tqdm import tqdm

from dotenv import load_dotenv
from openai import OpenAI
from prompts import sys_prompt

In [2]:
intents = []
for intent in os.listdir("/home/suchitg/amazon_help/leafdirs/"):
    if "IssueWith" not in intent:
        intents.append(intent)

In [3]:
len(intents)

53

In [4]:
len(os.listdir("/home/suchitg/amazon_help/leafdirs/"))

65

In [5]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [6]:
usr_prompts = []
for intent in intents:
    guidelines = open(f"/home/suchitg/amazon_help/leafdirs/{intent}/t.txt", "r").read()
    num_lines = len(open(f"/home/suchitg/amazon_help/leafdirs/{intent}/t.txt", "r").readlines())
    usr_prompts.append((
        f"""{{
    "title": "{intent}",
    "content": '''{guidelines}\n'''
}}""", num_lines))


In [7]:
def generate_queries(usr_prompt: tuple[str, int]):
    """Generate customer queries for the given intent and its guidelines."""
    res = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            # generate 10 queries if ground guideline document has > 15 lines
            # 5 otherwise
            {"role": "system", "content": sys_prompt.format(num_queries=15 if usr_prompt[1] > 15 else 7)},
            {"role": "user", "content": usr_prompt[0]},
        ],
        response_format={"type": "json_object"},
    )
    return res.choices[0].message.content


In [8]:
def remove_numbers(queries_obj: dict):
    """Remove numbers from the start of the queries."""
    updated = {
        k: [re.sub(r"^\d+\.\s", "", i) for i in v]
        for k, v in queries_obj.items()
        if k == "customer_queries"
    }
    queries_obj["customer_queries"] = updated["customer_queries"]

In [9]:
def main():
    f = open("./queries.jsonl", "a")
    for prompt in tqdm(usr_prompts):
        res = generate_queries(prompt)
        
        queries_obj = json.loads(res)
        queries_obj["tone"] = "neutral"
        queries_obj["style"] = "regular"

        json.dump(queries_obj, f, ensure_ascii=False)
        f.write("\n")
    f.close()

In [10]:
main()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [03:53<00:00,  4.41s/it]


In [11]:
n = 0
with open("queries.jsonl", 'r') as f:
    for json_str in f:
        obj = json.loads(json_str)
        n += len(obj['customer_queries'])

In [12]:
print(n)

715
